In [123]:
from flask import Flask, request, render_template, jsonify, json
from decouple import config
from flask_cors import CORS
import numpy as np
import networkx as nx
import tweepy
from sklearn.feature_extraction.text import CountVectorizer
import pandas as pd
import re
from collections import Counter
TWITTER_AUTH =tweepy.OAuthHandler(config('TWITTER_CONSUMER_KEY'),config('TWITTER_CONSUMER_SECRET'))
TWITTER_AUTH.set_access_token(config('TWITTER_ACCESS_TOKEN'),config('TWITTER_ACCESS_TOKEN_SECRET'))
TWITTER = tweepy.API(TWITTER_AUTH)


def get_first_user_interactions(search, index):
    twitter_user =TWITTER.get_user(search)
    tweets = twitter_user.timeline(
                      count =200,
                      exclude_replies = False,
                      include_rts=True,
                      tweet_mode='extended'
                      )
    #b is a transistion variable
    b = [ i.full_text for i in tweets ]
    b = " ".join(b)
    b = b.lower()
    b = b.replace(search, "")
    index =re.findall(r'(?<=^|(?<=[^a-zA-Z0-9-_\.]))@([A-Za-z]+[A-Za-z0-9-_]+)',b)

    return [ i[0] for i in Counter(index).most_common(10) ]

def get_first_user_connections(search, interactions_list):
    twitter_user =TWITTER.get_user(search)
    tweets = twitter_user.timeline(
                  count =200,
                  exclude_replies = False,
                  include_rts=True,
                  tweet_mode='extended'
                  )
    #b is a transistion variable
    b = [ i.full_text for i in tweets ]
    b = " ".join(b)
    b = b.lower()
    b = b.replace(search, "")
    interactions = re.findall(r'(?<=^|(?<=[^a-zA-Z0-9-_\.]))@([A-Za-z]+[A-Za-z0-9-_]+)',b)
    interactions = " ".join(interactions)
    interactions_list.append(interactions)
    return interactions_list

In [131]:
def get_network(user):
#     user = request.values["user"]
#     user = user.replace("@","")
    interactions_list = []
    index = []
    index =  get_first_user_interactions(user, index)
    index = list(set([search] + index))
    for i in index:
        get_first_user_connections(i, interactions_list)
    #add search term to front of index
    
    #turn the interactions into a dataframe

    vectorizer = CountVectorizer(min_df=2)
    X = vectorizer.fit_transform(interactions_list)
    display_df = pd.DataFrame(X.toarray(),columns=vectorizer.get_feature_names(), index=index)

    #create a graph and add edges
    DG = nx.MultiGraph()
    for i in display_df.index.values:
         for j in display_df.columns:
            if display_df[j].loc[i] > 0:
                DG.add_edge(j,i,weight = np.sqrt(display_df[j].loc[i] ))
            else:
                pass
    #create json nodes from graph
    sl_nodes = nx.spectral_layout(DG)
    edges = list(DG.edges.data())
    sl_edges_x = []
    sl_edges_y = []
    sl_edges_weight = []
    for i in range(len(edges)):
        sl_edges_x.append(sl_nodes[edges[i][0]][0])
        sl_edges_y.append(sl_nodes[edges[i][0]][1])
        sl_edges_x.append(sl_nodes[edges[i][1]][0])
        sl_edges_y.append(sl_nodes[edges[i][1]][1])
        sl_edges_weight.append(edges[i][2]["weight"])
        sl_edges_weight.append(edges[i][2]["weight"])
        sl_edges_x.append("")
        sl_edges_y.append("")
        sl_edges_weight.append("")
    x = json.dumps(sl_edges_x)
    y = json.dumps(sl_edges_y)
    weights = json.dumps(sl_edges_weight)
    return display_df

In [133]:
display_Df = get_network("austen")

In [136]:
display_Df["austen"].sum()

161

In [126]:
display_df

abdul_qadir41  akgrenier  alicejoyk  altimor  amyshackles  \
austenallred                0          0          0        0            0   
austen                      0          0          0        0            1   
trevmckendrick              0          1          1        2            0   
thebigknell                 4          0          0        0            1   
lambdaschool                0          0          0        0            1   
calebhicks                  1          1          2        0            0   
garybernhardt               0          0          0        0            0   
vboykis                     0          0          0        1            0   
vgr                         0          0          0        0            0   
litonico_                   0          0          0        0            0   

                anthonyjcamp  armadsen  at_neuner  austen  austenallred  \
austenallred               0         0          0       1             0   
austen                     0         0          1       0             0   
trevmckendrick             0         1          0       9             0   
thebigknell                1         6          1       4             4   
lambdaschool               1         0          1      61             0   
calebhicks                 0         4          0      13             9   
garybernhardt              0         0          0       0             0   
vboykis                    0         0          0       0             0   
vgr                        0         0          0       0             0   
litonico_                  0         0          0       0             0   

                billdifulvio  bitcoinbingbong  bjc290  calebhicks  \
austenallred               0                0       0           0   
austen                     0                5       1           0   
trevmckendrick             0                0       0           4   
thebigknell                4                0       1           9   
lambdaschool               1                1       0           1   
calebhicks                 0                0       1           0   
garybernhardt              0                0       0           0   
vboykis                    0                0       0           0   
vgr                        0                0       0           0   
litonico_                  0                0       0           0   

                ckiddwebdev  clervius  clintsmithiii  code  coetry  collision  \
austenallred              0         0              0     0       0          0   
austen                    0         0              0     0       0          0   
trevmckendrick            0         0              0     0       0          3   
thebigknell               1         2              0     1       7          0   
lambdaschool              0         1              0     0       1          0   
calebhicks                2         0              0     1       0          1   
garybernhardt             0         0              1     0       0          0   
vboykis                   0         0              0     0       0          0   
vgr                       0         0              0     0       0          0   
litonico_                 0         0              1     0       0          0   

                danielgross  daradoesdev  darienpayton  dustint314  erostribe  \
austenallred              0            0             0           0          0   
austen                    1            0             0           0          3   
trevmckendrick            0            1             0           0          0   
thebigknell               0            0             1           4          0   
lambdaschool              2            1             1           3          0   
calebhicks                0            0             0           1          1   
garybernhardt             0            0             0           0          0   
vboykis                   0            0  

In [101]:
a = display_df[0]
b = display_df[1]
len(a),len(b)

(10, 11)

In [ ]:
set(display_df["rrherr"].loc["austen"]

In [ ]:
 for i in display_df.index.values:
         for j in display_df.columns:
            if display_df[j].loc[i] > 0:
                #DG.add_edge(j,i,weight = np.sqrt(display_df[j].loc[i] ))
                print(display_df[j].loc[i])
            else:
                pass

In [76]:
pd.set_option("display.max_columns", 250)
display_df

abdul_qadir41  akgrenier  alicejoyk  altimor  amuellerml  \
austen                      0          0          0        0           1   
trevmckendrick              0          1          1        2           0   
austen                      0          0          0        0           0   
lambdaschool                0          0          0        0           0   
garybernhardt               0          0          0        0           0   
vgr                         0          0          0        0           0   
calebhicks                  1          1          2        0           0   
austenallred                0          0          0        0           0   
vboykis                     0          0          0        1           2   
thebigknell                 4          0          0        0           0   
litonico_                   0          0          0        0           0   

                amyshackles  anthonyjcamp  aparrish  armadsen  ashcraft_jj  \
austen                    0             0         1         1            1   
trevmckendrick            0             0         0         1            0   
austen                    1             0         0         0            0   
lambdaschool              1             1         0         0            0   
garybernhardt             0             0         1         0            0   
vgr                       0             0         0         0            0   
calebhicks                0             0         0         4            0   
austenallred              0             0         0         0            0   
vboykis                   0             0         0         0            0   
thebigknell               1             1         0         6            1   
litonico_                 0             0         0         0            0   

                at_neuner  austen  austenallred  bangbangconwest  \
austen                  0      12             8                1   
trevmckendrick          0       9             0                0   
austen                  1       0             0                0   
lambdaschool            1      61             0                0   
garybernhardt           0       0             0                0   
vgr                     0       0             0                0   
calebhicks              0      13             9                0   
austenallred            0       1             0                0   
vboykis                 0       0             0                0   
thebigknell             1       4             4                0   
litonico_               0       0             0                1   

                billdifulvio  bitcoinbingbong  bjc290  calebhicks  \
austen                     0                0       0           9   
trevmckendrick             0                0       0           4   
austen                     0                5       1           0   
lambdaschool               1                1       0           1   
garybernhardt              0                0       0           0   
vgr                        0                0       0           0   
calebhicks                 0                0       1           0   
austenallred               0                0       0           0   
vboykis                    0                0       0           0   
thebigknell                4                0       1           9   
litonico_                  0                0       0           0   

                chriscoyier  ckiddwebdev  clervius  clintsmithiii  code  \
austen                    1            0         0              0     0   
trevmckendrick            0            0         0              0     0   
austen                    0            0         0              0     0   
lambdaschool              0            0         1              0     0   
garybernhardt             0            0         0              1     0   
vgr                       0            0         0              0     

In [63]:
len(edges_dict)

257

In [64]:
unique_e = list({v['id']:v for v in edges_dict}.values())

In [66]:
len(unique_e)

247